In [2]:
import sys
sys.path.insert(0,'/home/kyle/GWA/NANOGrav/PsrSigSim/')
import psrsigsim as PSS
import numpy as np
import h5py
import os
import copy


#Bokeh imports
from bokeh.io import curdoc, output_file, show, output_notebook
from bokeh.layouts import column, row, widgetbox, gridplot
from bokeh.models import ColumnDataSource, Range1d, LinearColorMapper
import bokeh.models.widgets as widgets
from bokeh.plotting import figure

import matplotlib


output_notebook()

Loading BokehJS ...

In [21]:
psr_dict = {}
psr_dict['f0'] = 1400                   #Central frequency
psr_dict['F0'] = 218                    #Pulsar spin freq
psr_dict['bw'] = 400                    #Bandwidth
psr_dict['Nf'] = 512                    #Frequency bins
psr_dict['ObsTime'] = 1000/psr_dict['F0']  #Observation time
psr_dict['f_samp'] = 0.2                #Sampling frequency
psr_dict['SignalType'] = "intensity"    #'intensity' which carries a Nf x Nt
#filterbank of pulses or 'voltage' which carries a 4 x Nt array of
#voltage vs. time pulses representing 4 stokes channels
psr_dict['dm'] = 2                     #Dispersion Measure Pescs/(CM^3)
# V_ISS -- Intersteller Scintilation Velocity
psr_dict['scint_bw'] =  15.6            #Scintilation Bandwidth
psr_dict['scint_timescale'] = 2630      #Scintilation Timescale
# pulsar -- pulsar name
# telescope -- telescope name(GBT or Arecibo)
psr_dict['freq_band'] = 1400            #Frequency band [327 ,430, 820, 1400, 2300]
# aperature -- aperature (m)
# area -- collecting area (m^2)
# Tsys -- system temp (K), total of receiver, sky, spillover, etc. (only needed for noise)
# name -- GBT or Arecibo
# tau_scatter -- scattering time (ms)
psr_dict['radiometer_noise'] =  False   #radiometer noise
psr_dict['data_type']='float32'         #Was int8
psr_dict['flux'] = 3
psr_dict['to_DM_Broaden'] = False


#Constants for generating data--------------------------------------------------
dm_range = (0,10)
dm_range_spacing = 0.25
NumPulses = 1 #Don't change this. A bunch of stuff uses variables that depend on
#this being 1
startingPeriod = 0
start_time = (startingPeriod / psr_dict['F0']) *1000  #Getting start time in ms
TimeBinSize = (1.0/psr_dict['f_samp']) * 0.001
start_bin = int((start_time)/TimeBinSize)
stop_time = (((1 / psr_dict['F0']) *1000) * NumPulses) + start_time
# start_time + however many pulses times the pulsar period in ms
stop_bin =int((stop_time)/TimeBinSize)
first_freq = psr_dict['f0']-(psr_dict['bw']/2)
last_freq = psr_dict['f0']+(psr_dict['bw']/2)

foldingAdditionFactor = .1
FL_tau_scatter =0.005
FL_f0 = 1150
FL_bw = 1700
FL_Nf = 34     #Using using 34, the Bandwidth will make integers for steps in the slider

FL_dm = 0.001
FL_flux = 80



DMFullData = None
DM_list = list(np.arange(dm_range[0],dm_range[1]+dm_range_spacing,step=dm_range_spacing))

ScatterData = None

PreFoldingData = None
PostFoldingData = None

In [22]:
psr_dict['tau_scatter'] = FL_tau_scatter
psr_dict['f0'] = FL_f0
psr_dict['bw'] = FL_bw
psr_dict['Nf'] = FL_Nf
psr_dict['dm'] = FL_dm
psr_dict['radiometer_noise'] =  True
psr_dict['flux'] = FL_flux
psr_dict['to_Scatter_Broaden_exp'] = True

psr = PSS.Simulation(psr =  None , sim_telescope= 'GBT',
                     sim_ism= None, sim_scint= None,
                     sim_dict = psr_dict)

In [25]:
psr.init_signal()
psr.init_pulsar()
psr.init_ism()
psr.pulsar.gauss_template(peak=.5)
psr.init_telescope()
psr.simulate()
currentData = psr.obs_signal + foldingAdditionFactor*psr.signal.signal
PreFoldingData = np.copy((currentData[:,start_bin:stop_bin]))#Removed Swapaxes
PreFoldingData = np.reshape(PreFoldingData,(np.size(PreFoldingData)))
filterBank = psr.signal.signal[:,start_bin:stop_bin]

100% dispersed in 0.066 seconds.

/home/kyle/anaconda3/lib/python3.5/site-packages/mkl_fft/_numpy_fft.py:1044: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.rfftn_numpy(a, s, axes)


In [119]:
print(PreFoldingData.shape)
print(filterBank.shape)

(31178,)
(34, 917)


In [24]:
DMCM = LinearColorMapper(palette="Plasma256", low=0.0025, high=10)

DMsrc = ColumnDataSource(data=dict(image=[filterBank[:,:]],x=[start_time],y=[first_freq]))


DMfig = figure(title='Filter Bank',
             x_range = Range1d(start_time,stop_time),
             y_range = Range1d(first_freq,last_freq),
             x_axis_label = 'Observation Time (ms)',
             y_axis_label = 'Frequency (MHz)',
             tools = "crosshair,pan,reset,wheel_zoom")

DMfig.image(source = DMsrc,image='image',x='x', y='y',# image=[DMFullData[1,:,:]]
          dw=(stop_time-start_time), dh=(last_freq - first_freq),
          color_mapper = DMCM)
show(DMfig)

In [11]:
toShow = np.reshape(PreFoldingData,(34,917))
DMCM1 = LinearColorMapper(palette="Plasma256", low=0.0025, high=10)

DMsrc1 = ColumnDataSource(data=dict(image=[toShow[:,:]],x=[start_time],y=[first_freq]))


DMfig1 = figure(title='Filter Bank',
             x_range = Range1d(start_time,stop_time),
             y_range = Range1d(first_freq,last_freq),
             x_axis_label = 'Observation Time (ms)',
             y_axis_label = 'Frequency (MHz)',
             tools = "crosshair,pan,reset,wheel_zoom")

DMfig1.image(source = DMsrc1,image='image',x='x', y='y',
          dw=(stop_time-start_time), dh=(last_freq - first_freq),
          color_mapper = DMCM1)

show(DMfig1)

In [129]:
def calcFolding(freq):
    global PostFoldingData
    PostFoldingData = None
    foldBin = int((1.0/freq)*1000/TimeBinSize) #length of period in terms of time bins
    height = int(PreFoldingData.size/foldBin)
    temp = np.copy(PreFoldingData)
    temp = np.resize(temp,(height,foldBin))
    PostFoldingData = np.sum(temp,axis=0)


In [103]:
print(TimeBinSize)

0.005


In [106]:
def write():
    global PreFoldingData
    if(os.path.exists('testing.hdf5')):
        os.remove('testing.hdf5')

    f = h5py.File('testing.hdf5','w')
    f.create_dataset('test',data=PreFoldingData)
    f.close()
    PreFoldingData = None

def read():
    global PreFoldingData
    f = h5py.File('testing.hdf5','r')
    PreFoldingData = np.array(f.get('test'),copy=True)
    f.close()

In [135]:
preWR_preFold = copy.deepcopy(PreFoldingData)
calcFolding(218)
showPlot()
preWR_postFold =copy.deepcopy(PostFoldingData)
write()
read()
postWR_preFold = copy.deepcopy(PreFoldingData)
calcFolding(228)
showPlot()
postWR_postFold = copy.deepcopy(PostFoldingData)
print(np.array_equal(preWR_preFold,postWR_preFold))
print(np.array_equal(preWR_postFold,postWR_postFold))


New array will be: (917,34)
Copied Prefold data.
Resized Postfold data into:
[[0.00203476 0.10759095 0.27389032 ... 0.04779398 0.09323407 0.04953963]
 [0.03368553 0.18709206 0.3474785  ... 1.01423343 0.08210364 0.06343289]
 [0.02495667 0.00230162 0.64337345 ... 0.00656104 0.01138335 0.4837988 ]
 ...
 [0.09621237 0.36671382 0.12599962 ... 0.50181412 0.35211534 0.38917505]
 [0.27801816 0.34370021 0.04090966 ... 0.00759214 0.20183438 0.01436122]
 [0.13707179 0.03253516 0.38564582 ... 0.02324749 0.25616817 0.42241064]]


New array will be: (877,35)
Copied Prefold data.
Resized Postfold data into:
[[0.00203476 0.10759095 0.27389032 ... 0.04309626 0.1511809  0.40366382]
 [0.02382509 0.28644363 0.08867111 ... 0.00326843 0.08667758 0.0351848 ]
 [0.08326017 0.20858204 0.05586476 ... 0.00874146 0.34352161 0.0047452 ]
 ...
 [0.34667512 0.05493958 0.37980561 ... 0.12959124 0.0417913  0.18760215]
 [0.05704195 0.30491435 0.48908116 ... 0.2840019  0.14660874 0.47228991]
 [0.08372644 0.45395934 0.29010901 ... 0.24510652 0.27309522 0.18825691]]


True
False


In [98]:
def showPlot():
    #To check work
    src2 = ColumnDataSource(data = dict(x = np.linspace(0,1,PostFoldingData.shape[0]), 
                                        y = PostFoldingData ) )
    fig2 = figure(plot_width = 400, plot_height = 400, 
                  #x_range = Range1d(start_time,stop_time), 
                  y_range = Range1d(0,40),
                  x_axis_label = 'Pulse Phase',
                  y_axis_label = 'Pulse Intensity',)

    fig2.line(source = src2, x='x', y='y',)

    show(fig2)